In [1]:
import pandas as pd
import numpy as np

## Nasser's cleaning -- Weather data

In [2]:
cincinnati = pd.read_csv('cincinnati_data.csv').set_index('DATE').fillna(method='ffill')
baltimore = pd.read_csv('baltimore_data.csv').set_index('DATE').fillna(method='ffill')
richmond = pd.read_csv('richmond_data.csv').set_index('DATE').fillna(method='ffill')
cincinnati.index = pd.to_datetime(cincinnati.index).round('h')
baltimore.index = pd.to_datetime(baltimore.index).round('h')
richmond.index = pd.to_datetime(richmond.index).round('h')

columns = []
for i in cincinnati.columns:
    if 'Hourly' in i:
        columns.append(i)
        
cincinnati = cincinnati[columns]
baltimore = baltimore[columns]
richmond = richmond[columns]


columns.remove('HourlySkyConditions')
columns.remove('HourlyPresentWeatherType')
columns.remove('HourlyPrecipitation')

for col in columns:
    for i in enumerate(cincinnati[col]):
        if type(i[1]) == int or type(i[1]) == float:
            pass
        elif 's' in i[1]:
            cincinnati[col][cincinnati[col]==i[1]] = float(i[1][:-1])
            
for col in columns:
    for i in enumerate(baltimore[col]):
        if type(i[1]) == int or type(i[1]) == float:
            pass
        elif 's' in i[1]:
            baltimore[col][baltimore[col]==i[1]] = float(i[1][:-1])
            
for col in columns:
    for i in enumerate(richmond[col]):
        if type(i[1]) == int or type(i[1]) == float:
            pass
        elif 's' in i[1]:
            richmond[col][richmond[col]==i[1]] = float(i[1][:-1])
   

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3,30,31,32,41,42,43,44,49,51,58,69,71,72,88,95,96,97,98,99,100,101,102,103,104,105,106,107) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (30,31,33,34,41,42,43,49,51,69,71,75,76,88,89) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (30,31,33,34,35,36,41,42,43,48,49,51,53,56,58,59,69,71,73,75,76,88,89) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-2-c852634a9633>:27: SettingWithCopyWarning: 


## Index Dataframes in uniform manner, create general column names

In [3]:
ev = pd.read_excel('Solar_GenX.xlsx').set_index('datetime_beginning_ept').fillna(method='bfill') ## read in solar data
ev.index = pd.to_datetime(ev.index)                                                                 
#########################################################################################          
demand = pd.read_csv('Demand_data.csv',index_col='datetime_beginning_utc',parse_dates=True) ## read in demand data
ev1 = ev.iloc[:len(demand),[0,1,2,3]]                                                        ## make demand and solar df's the same length
ixx = ev1.index.to_list()
demand['datetime_beginning_ept'] = pd.Series(ixx, index=demand.index)     
demand = demand.set_index('datetime_beginning_ept')                      ### demand data had utc time, so converted to ept using solar df index
######################################################################################### 
WIND = pd.read_excel('WindGenDataFinal.xlsx',index_col='datetime_beginning_ept',parse_dates=True) 
WIND = WIND['2019-01-01 00:00:00':] ## start at same time as others
demand = demand['2019-01-01 00:00:00':'2020-11-08 18:00:00'] ## make demand end at same time as wind
ev1= ev1['2019-01-01 00:00:00':'2020-11-08 18:00:00'] ## make solar end at same time as wind
WIND = WIND.rename(columns={"MIDATL": "MIDATL_wind", "SOUTH": "SOUTH_wind","WEST": "WEST_wind"})
demand = demand.rename(columns={"MIDATL_DEMAND": "MIDATL_demand", "SOUTH_DEMAND": "SOUTH_demand","WEST_DEMAND": "WEST_demand"})

## Merge & create dataframes on region: WEST, MIDATL, SOUTH

In [4]:
WEST,MIDATL,SOUTH = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]
regions = ['WEST','MIDATL','SOUTH']
cities = [cincinnati,baltimore,richmond]
l = [WEST,MIDATL,SOUTH]
for i in range(len(regions)): #for each city or region
    df = pd.merge(cities[i],ev1[str(regions[i])+'_solar'],left_index=True, right_index=True)   # merge weather with solar
    df = pd.merge(df,WIND[str(regions[i])+'_wind'],left_index=True, right_index=True) # merge previous with wind
    df = pd.merge(df,demand[str(regions[i])+'_demand'],left_index=True, right_index=True) # merge previous with demand
    df = df.drop(['HourlyPresentWeatherType', 'HourlySkyConditions', 
                 'HourlyAltimeterSetting', 'HourlyPressureChange', 'HourlyPrecipitation',
                  'HourlyPressureTendency', 'HourlySeaLevelPressure', 'HourlyWetBulbTemperature'], axis=1)
    l[i] = df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index').sort_index()
    for col in l[i].columns:
        l[i][col] = (pd.to_numeric(l[i][col], errors='coerce').fillna(0))

WEST,MIDATL,SOUTH = l

In [7]:
WEST.to_excel("WEST.xlsx")  
SOUTH.to_excel("SOUTH.xlsx")  
MIDATL.to_excel("MIDATL.xlsx")  